![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Strategy 1: Buy and hold

Buy and hold generates profit from the risk premia of market risk exposition.

A buy and hold strategy of 1 instrument is the most simple strategy. It requires several considerations that will be touched on in this notebook

## Data aquisition

In [1]:

from datetime import datetime, timedelta
from QuantConnect import Resolution
from QuantConnect.Securities import Futures
from helper import calculate_return, calculate_roll_date, expiration_dates

# Initialize QuantBook
qb = QuantBook()

# Add Future Subscriptions for ES and MES with Daily resolution
es_future = qb.AddFuture(Futures.Indices.SP500EMini, Resolution.Daily)
mes_future = qb.AddFuture(Futures.Indices.MicroSP500EMini, Resolution.Daily)

# Set Historical Data Range
start_date = datetime(2007, 1, 1)
end_date = datetime.now()

# Request Historical Data for ES and MES futures
es_history = qb.History([es_future.Symbol], start_date, end_date, Resolution.Daily)
mes_history = qb.History([mes_future.Symbol], start_date, end_date, Resolution.Daily)

# Save data to Object Store (optional)
# qb.ObjectStore.Save("ES_Future_Data", es_history.to_csv(index=False))
# qb.ObjectStore.Save("MES_Future_Data", mes_history.to_csv(index=False))

es_history.reset_index(level=0, drop=True, inplace=True)
mes_history.reset_index(level=0, drop=True, inplace=True)

# Display the first few rows of the historical data
print("ES Future Data:")
print(es_history.head())

print("\nMES Future Data:")
print(mes_history.head())



ES Future Data:
                               askclose      askhigh       asklow  \
symbol time                                                         
ES 1S1 2007-01-03 19:00:00  1380.443221  1394.484909  1371.969788   
       2007-01-04 19:00:00  1383.348398  1386.253575  1372.211887   
       2007-01-05 19:00:00  1371.727690  1383.348398  1369.790906   
       2007-01-08 19:00:00  1378.022240  1379.474829  1368.580415   
       2007-01-09 19:00:00  1376.327554  1379.959025  1369.548808   

                                askopen  asksize     bidclose      bidhigh  \
symbol time                                                                  
ES 1S1 2007-01-03 19:00:00  1388.674556    173.0  1380.201123  1394.242811   
       2007-01-04 19:00:00  1380.443221    510.0  1383.106300  1386.011477   
       2007-01-05 19:00:00  1383.348398     30.0  1371.243494  1383.106300   
       2007-01-08 19:00:00  1371.727690    103.0  1377.780142  1379.232731   
       2007-01-09 19:00:00  1378

## Multipliers and tick size

Each contract may have different multipliers. That means, that a 1$ move is equal to a 50$ change in returns for each contract one is exposed to. We will create a dictionary with the relevant information to calculate the returns down the road.

### Compute Functions

In [2]:
instruments = {
    'ES': {
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['ES']
    },
    'MES': {
        'multiplier': 5,
        'tick_value': 0.25,
        'minimum_fluctuation': 1.25,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['MES']
    }
}


In [3]:
def simulate_buy_and_hold(history_df, symbol):
    """
    Simulate buy-and-hold strategy. Takes historical price DataFrame and symbol.
    Returns net total return after accounting for transaction costs.
    """
    params = instruments[symbol]
    expiration_dates = params['expiration_dates']
    multiplier = params['multiplier']
    spread = params['spread']
    commission = params['commission']
    
    initial_price = history_df['close'].iloc[0]
    final_price = history_df['close'].iloc[-1]
    total_return = calculate_return(symbol, initial_price, final_price, instruments)
    
    # Calculate transaction costs
    transaction_costs = 0
    for expiration_date in expiration_dates:
        roll_date = calculate_roll_date(expiration_date)
        if start_date <= roll_date <= end_date:
            transaction_costs += ((spread * multiplier) + commission) * 2
    
    total_return -= transaction_costs
    return total_return


In [4]:

# Calculate buy and hold returns for ES and MES
retorno_es = simulate_buy_and_hold(es_history, 'ES')
retorno_mes = simulate_buy_and_hold(mes_history, 'MES')

print(f"Retorno total ES (con costos de transacción): {retorno_es}")
print(f"Retorno total MES (con costos de transacción): {retorno_mes}")


Retorno total ES (con costos de transacción): 194762.44385132618
Retorno total MES (con costos de transacción): 11211.04044389167
